In [1]:
import findspark
from pyspark.sql import SparkSession
import pyspark.sql.types as T
from delta import *

findspark.init()

In [2]:
builder = SparkSession \
    .builder \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.debug.maxToStringFields", "1000") \
    .appName("notebook")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

23/10/31 09:26:12 WARN Utils: Your hostname, Capotes-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.140.103 instead (on interface en0)
23/10/31 09:26:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/rafael/personal/spark-playground/stateful-streaming/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/rafael/.ivy2/cache
The jars for the packages stored in: /Users/rafael/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d6d66cc2-6dbb-4c3d-8445-3ec788cfee2e;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 186ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [3]:
df = spark.read.format("delta").load("./table")

In [4]:
df.orderBy("end_ts").toPandas()

/Users/rafael/personal/spark-playground/stateful-streaming/.venv/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/Users/rafael/personal/spark-playground/stateful-streaming/.venv/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,sensor_id,count,threshold,triggered,start_ts,end_ts,delta
0,XYZ,2,2,True,2023-10-26 14:15:24.681721,2023-10-26 14:15:38.444520,13.762799
1,XYZ,2,2,True,2023-10-26 14:17:08.296380,2023-10-26 14:17:32.098566,23.802186


In [5]:
spark.stop()